# "Live" Tweets from Stream API using Tweepy Library <br>

_Authors: Amy Taylor and Veronica Giannotta_ <br>


Preliminary steps before using Twitter's API:
1. Sign-up for a twitter account
2. Register a twitter developer account (requires email or phone number)
3. Create a developer app (I went with the name BlockedRoads)
4. Obtain your 'Access token' and 'access token secret' in the developer dashboard

## Part 1: Pull Live Tweets from Twitter using Tweepy and Streaming API - StreamListener

Some resources:
http://socialmedia-class.org/twittertutorial.html
<br>https://www.dataquest.io/blog/streaming-data-python/


**Step 1: In terminal run:** `! pip install tweepy`

**Step 2: Load imports**

In [2]:
import pandas as pd
import tweepy
import json
import datetime
from tweepy.streaming import StreamListener
import shutil
import config

**Step 2: Authenticate Twitter Credentials through Tweepy**

In [4]:
# authenticate account with tweepy
auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_key, config.access_secret)

# create API object to pull data from twitter - and pass in code!
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# now we should be free to make twitter api calls!

**Step 3: Verify API is working with your account**

In [5]:
# check if program is working - output is your twitter name
user = api.me()
print (user.name)

amy taylor


In [5]:
# To send a Tweet from your account (assuming you allowed your app to have read/write permissions):
# api.update_status(status = "Hey, I'm tweeting with Tweepy!")

**Step 4: Instantiate the StreamListener class and start pulling tweets**
- Specify how tweets are downloaded inside the `on_status` or `on_data` class method
- `stream.filter` starts the download. Specify your search words with the `track` parameter
- To stop the live stream, **kernel interrupt the codeblock**
- Continue to Part 2 to save the tweets into a json file

In [ ]:
class StreamListener(tweepy.StreamListener):            
    def on_data(self, status):   
        try:
            with open('../data/AT_stream/scrape.json', 'a') as f:
                f.write(status)
        except:
            print("Could not open file log")
        
         
    def on_error(self, status_code):
        if status_code == 420:
            return False

        
stream_listener = StreamListener()
stream = tweepy.Stream(auth=auth, listener= stream_listener)
# starts the stream
stream.filter(track=[ "road closed", "roads closed", "road blocked", "street blocked", "roadclosed"])

## Part 2: Save Tweet to a JSON file

In [11]:
def file_conversion():
    #create a timestamp
    now = datetime.datetime.now()
    month = '0'+str(now.month)
    day = str(now.day)
    hour = str(now.hour)
    minute = str(now.minute)
    now_str = str(now.year)+month+day+'_'+hour+'_'+minute
    
    
    #replace the name of our file with a new timestamped filename
    dest = '../data/AT_stream/scrape_' + now_str + ".json"
    shutil.move('../data/AT_stream/scrape.json', dest)
    
    with open(dest, "r") as f:
        status = f.readlines()
        jsons = []
        for ind in status:
            jsons.append(json.loads(ind))
    return jsons

# Uncomment out the file_conversion function to activate the file name change
# file_conversion()

**Table of most recent stream downloads**

|Stream| params| json file| # of tweets |search terms|
|---| --- | --- | --- | ---|
|1| tweet_mode='extended'| './scrape_20190115_11_28.json' | 1| track1|
|2| none?| './scrape_20190116_12_15.json'  | 4| track1|

track1 =[ "road closed", "roads closed", "road blocked", "street blocked", "roadclosed"]


In [6]:
# Previously created json files
# json_df = pd.read_json("../data/AT_stream/scrape_20190116_11_28.json", lines = True)
json_df = pd.read_json("../data/AT_stream/scrape_20190116_12_15.json", lines = True)

In [7]:
print(json_df.shape)
json_df.head()

(4, 32)


,contributors,coordinates,created_at,entities,favorite_count,favorited,filter_level,geo,id,id_str,...,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,NaN,NaN,2019-01-16 20:14:10,"{'user_mentions': [], 'symbols': [], 'hashtags...",0,False,low,NaN,1085631317751066626,1085631317751066624,...,"{'display': 'twitter.com/kil889/status/…', 'ex...",0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Just block anybody who willingly listens to St...,2019-01-16 20:14:10.863,False,{'name': 'Tragedy Khadafi verse on Strange Fru...
1,NaN,NaN,2019-01-16 20:14:13,{'user_mentions': [{'name': 'Black Nerd Proble...,0,False,low,NaN,1085631330384334848,1085631330384334848,...,"{'display': 'twitter.com/rhyemswturtle/…', 'ex...",0,0,False,{'created_at': 'Wed Jan 16 18:59:47 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...","RT @BlkNrdProblems: Beloved, that's called str...",2019-01-16 20:14:13.875,False,"{'name': 'lmao y'all wild', 'screen_name': 'An..."
2,NaN,NaN,2019-01-16 20:14:15,"{'user_mentions': [], 'symbols': [], 'hashtags...",0,False,low,NaN,1085631338693214208,1085631338693214208,...,NaN,0,0,False,NaN,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",TFL UPDATE: TfLTrafficNews: London Road SM4 is...,2019-01-16 20:14:15.856,False,"{'name': 'Bob', 'screen_name': 'thelondonbob01..."
3,NaN,NaN,2019-01-16 20:14:51,{'user_mentions': [{'name': 'Highways England'...,0,False,low,NaN,1085631487955869697,1085631487955869696,...,NaN,0,0,False,"{'truncated': True, 'created_at': 'Wed Jan 16 ...","<a href=""http://twitter.com/download/android"" ...",RT @HighwaysNWEST: All traffic caught within t...,2019-01-16 20:14:51.443,False,"{'name': 'Cumbria charity truck fair', 'screen..."


## Part 3: Unpack the nested dictionary into separate columns

In [8]:
tweets_data = []
notParsed = []
tweets_file = open("../data/AT_stream/scrape_20190116_12_15.json","r")
for line in tweets_file:    
    if line.strip():    
        try:
            tweet=json.loads(line)
            tweets_data.append(tweet)
        except:
            notParsed.append(line)
            continue
print(len(tweets_data))
print('Could not parse: ', len(notParsed))

4
Could not parse:  0


In [10]:
tweet_cols = ['coordinates', 'created_at', 
#                'full_text',
              'text','geo', 'id', 'place', 'user']


sample_tweets_dict = [{col:tweet[col] for col in tweet_cols } for tweet in tweets_data]

# This method looks for any instances of nested dictionaries (DOES NOT FLATTEN LISTS)
sample_tweets_df = pd.io.json.json_normalize(sample_tweets_dict)
sample_tweets_df

,coordinates,created_at,geo,id,place,text,user.contributors_enabled,user.created_at,user.default_profile,user.default_profile_image,...,user.profile_text_color,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.translator_type,user.url,user.utc_offset,user.verified
0,None,Wed Jan 16 20:14:10 +0000 2019,None,1085631317751066626,None,Just block anybody who willingly listens to St...,False,Sat Apr 03 01:38:23 +0000 2010,False,False,...,333333,True,False,MiqueOnTheMix,138078,None,none,http://Soundcloud.com/miqueonthemix,None,False
1,None,Wed Jan 16 20:14:13 +0000 2019,None,1085631330384334848,None,"RT @BlkNrdProblems: Beloved, that's called str...",False,Sat Oct 24 15:46:11 +0000 2015,False,False,...,000000,False,False,AnAcidTweeter,1833,None,none,None,None,False
2,None,Wed Jan 16 20:14:15 +0000 2019,None,1085631338693214208,None,TFL UPDATE: TfLTrafficNews: London Road SM4 is...,False,Mon Jul 04 20:11:00 +0000 2016,True,False,...,333333,True,False,thelondonbob01,104997,None,none,None,None,False
3,None,Wed Jan 16 20:14:51 +0000 2019,None,1085631487955869697,None,RT @HighwaysNWEST: All traffic caught within t...,False,Sat Jul 23 20:06:00 +0000 2016,True,False,...,333333,True,False,Cctf2,3572,None,none,https://www.facebook.com/CumbriaCharityTruckFair/,None,False


Let's look at the tweets we collected during the live stream

In [11]:
list = json_df.loc[:, 'text']
# print(list)
for i in range(len(list)):
    print(i, list[i])
    print("--------")

0 Just block anybody who willingly listens to Streetlife
--------
1 RT @BlkNrdProblems: Beloved, that's called street harassment. Catcalling. Unwanted attention from dudes on the street that NO WOMAN WANTS.…
--------
2 TFL UPDATE: TfLTrafficNews: London Road SM4 is closed in both directions due to a road traffic collision.
--------
3 RT @HighwaysNWEST: All traffic caught within the closure has now been released. Thanks for your patience if you were held up. The road rema…
--------
